In [37]:
import java.io.*
import java.nio.charset.Charset

fun readAndMap(file: String): List<String> {
    val lines = File("../data/in/$file").readLines(Charsets.UTF_8)
    val relevantLines = lines.filter { it.startsWith("# newdoc") || it.startsWith("# text") }
    return relevantLines
        .map {
            if (it.startsWith("# newdoc")) {
                ""
            } else if (it.startsWith("# text")) {
                it.replace("# text = ", "")
            } else {
                throw RuntimeException("this should not exist: $it")
            }
        }
        // Remove initial empty line
        .drop(1)
}


In [38]:
fun sentsToFile(sentences: List<String>, file: String): Unit {
    val fileWriter = FileWriter("../data/out/$file")
    fileWriter.use {
        sentences.forEach {
            fileWriter.write(it + System.lineSeparator())
        }
    }
}

In [39]:
val sentences = readAndMap("sv_talbanken-ud-train.conllu")
sentsToFile(sentences, "sv_talbanken-se-sent.train")

In [40]:
val sentences = readAndMap("sv_talbanken-ud-test.conllu")
sentsToFile(sentences, "sv_talbanken-se-sent.eval")
